[![Radiant MLHub Logo](https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png)](https://mlhub.earth/)

# Models

A **Model** in the Radiant MLHub API is a STAC item implementing the [ML Model
extension](https://github.com/stac-extensions/ml-model). The goal of the STAC
ML Model Extension is to provide a way of cataloging machine learning (ML)
models that operate on Earth observation (EO) data described as a STAC catalog.

This notebook will walk you through a few common techniques for working with
Radiant MLHub model, including:

* Listing available models
* Fetching a model
* Fetching assets and links associated with a model




### Import Libraries

In [1]:
from pprint import pprint

from radiant_mlhub import MLModel, client

## `MLModel` Class

Using the `radiant_mlhub.MLModel` class is the recommended method for working with models as Python objects (see [Low-Level Client](#Low-Level-Client) docs below for how to work with raw API responses). The `MLModel` class has some convenient methods for listing and fetching models, as well as fetching the assets associated with those models.

### List Models

You can use the `MLModel.list` class method to list all models available
through the Radiant MLHub API. Each instance has `id` and `properties` attributes
that you can inspect to get more information about the model, and the `assets` and
`links` properties that you can use to get the links associated with the
models.

In [2]:
models = MLModel.list()

# List the title and ID of first 10 models returned by the API
for model in models:
    print(f"{model.properties.get('title')} ({model.id})")

Tropical Cyclone Wind Estimation Model (model-cyclone-wind-estimation-torchgeo-v1)


### Fetch a Model

If you know the ID of a model, you can also fetch it directly using the `MLModel.fetch` method. This method returns a `MLModel` instance.

In [3]:
cyclone_model = MLModel.fetch('model-cyclone-wind-estimation-torchgeo-v1')
print(cyclone_model.properties.get('description'))

This is a PyTorch model trained on the [Tropical Cyclone Wind Estimation Competition](https://registry.mlhub.earth/10.34911/rdnt.xs53up/) dataset with v0.1 of the [TorchGeo](https://github.com/microsoft/torchgeo) package. The model is a resnet18 model pretrained on [ImageNet](https://www.image-net.org/index.php) then trained with a MSE loss. The data were randomly split 80/20 by storm ID and an early stop was used based on performance.


### List Model's Assets and Links

A Model has assets, e.g. model inferencing runtime, model checkpoint, and also links
for example to training data, and/or to related Datasets.

In [4]:
for link in cyclone_model.links:
    print(f"{link.rel} link -> {link.href}")

collection link -> http://api.radiant.earth/mlhub/v1/collections/model-cyclone-wind-estimation-torchgeo
parent link -> http://api.radiant.earth/mlhub/v1/collections/model-cyclone-wind-estimation-torchgeo
root link -> http://api.radiant.earth/mlhub/v1/
self link -> http://api.radiant.earth/mlhub/v1/collections/model-cyclone-wind-estimation-torchgeo/items/model-cyclone-wind-estimation-torchgeo-v1
ml-model:inferencing-image link -> docker://docker.io/radiantearth/cyclone-model-torchgeo:1
ml-model:train-data link -> https://api.radiant.earth/mlhub/v1/collections/nasa_tropical_storm_competition_train_source
mlhub:training-dataset link -> https://mlhub.earth/data/nasa_tropical_storm_competition


In [5]:
# note: the assets property is a dictionary
for key, asset in cyclone_model.assets.items():
    print(f"{key}: {asset.title} {asset.href}")

inferencing-compose: Model inferencing runtime https://raw.githubusercontent.com/RadiantMLHub/cyclone-model-torchgeo/main/inferencing.yml
inferencing-checkpoint: Final model checkpoint https://zenodo.org/record/5773331/files/last.ckpt?download=1


## Low-Level Client

The low-level client functions also provide a way of interacting with the Radiant MLHub API `/models` endpoint using Python. These methods return native Python data types (e.g. `list`, `dict`, etc.) rather than the `MLModel` instances documented above.

All low-level client functions are contained in the `radiant_mlhub.client` module (imported above). All of these methods accept the `profile` and `api_key` keyword arguments, which are passed directly to `radiant_mlhub.get_session`, if provided.

### List Models

You can use the `list_models` method to loop through all of the available models. This method makes requests to the `/models` endpoint.

In [6]:
models = client.list_models()

first_model = models[0]
pprint(first_model['properties'])

{'datetime': None,
 'description': 'This is a PyTorch model trained on the [Tropical Cyclone Wind '
                'Estimation '
                'Competition](https://registry.mlhub.earth/10.34911/rdnt.xs53up/) '
                'dataset with v0.1 of the '
                '[TorchGeo](https://github.com/microsoft/torchgeo) package. '
                'The model is a resnet18 model pretrained on '
                '[ImageNet](https://www.image-net.org/index.php) then trained '
                'with a MSE loss. The data were randomly split 80/20 by storm '
                'ID and an early stop was used based on performance.',
 'end_datetime': '9999-12-31T23:59:59Z',
 'license': 'MIT',
 'ml-model:architecture': 'resnet18',
 'ml-model:learning_approach': 'supervised',
 'ml-model:prediction_type': 'regression',
 'ml-model:type': 'ml-model',
 'providers': [{'email': 'caleb.robinson@microsoft.com',
                'name': 'Microsoft AI for Good Research Lab (Caleb Robinson)',
                'r

### Fetch a Model

You can use the `get_model_by_id` method to fetch a model by ID. This method returns a Python dictionary representing the model object.

In [7]:
cyclone_model_dict = client.get_model_by_id('model-cyclone-wind-estimation-torchgeo-v1')
pprint(cyclone_model_dict)

{'assets': {'inferencing-checkpoint': {'href': 'https://zenodo.org/record/5773331/files/last.ckpt?download=1',
                                       'roles': ['ml-model:checkpoint'],
                                       'title': 'Final model checkpoint',
                                       'type': 'application/octet-stream'},
            'inferencing-compose': {'href': 'https://raw.githubusercontent.com/RadiantMLHub/cyclone-model-torchgeo/main/inferencing.yml',
                                    'roles': ['ml-model:inference-runtime'],
                                    'title': 'Model inferencing runtime',
                                    'type': 'text/x-yaml; '
                                            'application=compose'}},
 'bbox': [-179.999, -89.999, 179.999, 89.999],
 'collection': 'model-cyclone-wind-estimation-torchgeo',
 'geometry': {'coordinates': [[[-179.999, -89.999],
                               [179.999, -89.999],
                               [179.999, 